
<a id='chap-tpdeeplearning2'></a>

# Travaux pratiques - Perceptron multi-couche

L'objectif de cette séance de travaux pratiques est de faire évoluer le modèle de régression logistique utilisé dans le précédent TP vers un réseau de neurones plus sophistiqué. Dans le TP précédent, nous avons travaillé avec la régression logistique, un modèle linéaire simple appartenant à la famille des réseaux de neurones.

Dans ce TP, nous allons nous attaquer à l'implémentation d'un perceptron multicouche (MLP - Multi-Layer Perceptron). Contrairement à la régression logistique, qui se limite à des séparations linéaires, le perceptron multicouche a la capacité d'apprendre des frontières de décision non linéaires. De plus, les perceptrons sont considérés comme des approximants universels pour les fonctions continues, ce qui signifie qu'ils sont extrêmement puissants et forment la base de l'apprentissage profond.

Nous commencerons par étudier l'implémentation de la phase de propagation avant (forward pass) pour effectuer des prédictions, puis la phase de rétropropagation (backward pass) pour entraîner un perceptron à une couche cachée.

Passons maintenant à la recharge du jeu de données MNIST :

In [120]:
import tensorflow as tf
from keras import backend as K
import keras
import numpy as np

In [121]:
# Import de MNIST depuis Keras
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()
# Transformation des images 28x28 en vecteur de dimension 784
X_train = X_train.reshape(60000, 784).astype('float32')
X_test = X_test.reshape(10000, 784).astype('float32')
# Normalisation entre 0 et 1
X_train /= 255
X_test /= 255

X_train.shape

(60000, 784)

## Prédiction avec un perceptron (*forward*)

L’architecture du perceptron à une seule couche cachée est illustrée dans la
figure ci-dessous.

Comme la dernière fois, on considère les données de la base MNIST. Chaque image est
représentée par un vecteur de taille $ 28^2=784 $. Le perceptron applique
différentes opérations mathématiques pour transformer l’entrée et produire
la prédiction finale, c’est-à-dire la catégorie de l’image :

1. Une projection linéaire, qui va projeter chaque image sur un vecteur de dimensions $ (1, L) $. $ L $ représente ici la largeur (le nombre de neurones) de la couche cachée du perceptron, par exemple $ L=100 $.
  En considérant que chaque exemple $ \mathbf{x_i} $ est un vecteur ligne $ (1,784) $, la projection linéaire est représentée par une matrice $ \mathbf{W^h} $ $ (784, L) $ et un vecteur de biais $ \mathbf{b^h} $
  $ (1, L) $. La projection s’écrit:  
  $$
  \mathbf{u_i} = \mathbf{x_i} \mathbf{W^h} + \mathbf{b^h}.
  $$
1. L’application d’une fonction de transfert non-linéaire, par exemple une sigmoïde :  
  $$
  \forall j \in \left\lbrace 1; L \right\rbrace, ~ h_{i,j} = \frac{1}{1+\exp(-u_{i,j})}.
  $$
1. Une deuxième projection linéaire, qui va projeter la représentation interne (les activations de la couche cachée) de dimensions $ (1,L) $ en un vecteur de $ (1, K) $, avec $ K $ le nombre de classes considérées (ici, 10). $ K $ représente le nombre de neurones en sortie, c’est-à-dire la dimensionalité du vecteur prédit. Cette opération de projection linéaire est représentée par la matrice $ \mathbf{W^y} $ de dimensions $ (L, K) $ et le vecteur de biais $ \mathbf{b^y} $ de dimensions $ (1, K) $). Matriciellement, la projection est représentée par l’opération :  
  $$
  \mathbf{v_i} =\mathbf{h_i} \mathbf{W^y} + \mathbf{b^y}.
  $$
1. Enfin, l’application d’une non-linéarité *softmax*. Comme pour la régression logistique, cela permet de transformer les activations de sortie en probabilités pour une distribution catégorielle :  
  $$
  \forall j \in \left\lbrace 1; K \right\rbrace ~ y_{i,j} = \frac{\exp(v_{i,j})}{\sum\limits_{k=1}^K \exp(v_{i,k})}.
  $$


Notre objectif pour cette séance va être d’implémenter un perceptron (et son apprentissage) sur la base MNIST. Commençons par transformer les étiquettes en vecteur encodé au format *one-hot*:

In [122]:
from keras.utils import to_categorical

n_classes = 10
# Conversion des étiquettes au format one-hot
Y_train = to_categorical(y_train,n_classes)
Y_test = to_categorical(y_test, n_classes)

Y_train.shape

(60000, 10)

## Question

En reprenant le squelette du code de la régression logistique, compléter le code ci-dessous pour implémenter la *forward pass* (phase de prédiction) du perceptron multi-couche.

Vous devrez notamment écrire une fonction `forward(batch, Wh, bh, Wy, by)` qui renvoie la prédiction $ \hat{\mathbf{y}} $ ainsi que la matrice des activations de la couche cachée.
Si l’on considère un batch des données de taille $ n \times 784 $, les paramètres $ \mathbf{W^h} $ ($ 784\times L $), $ \mathbf{b^h} $ ($ 1\times L $),
$ \mathbf{W^y} $($ L\times K $) et $ \mathbf{b^y} $ ($ 1\times K $), la fonction `forward` renvoie :

- la prédiction $ \mathbf{\hat{Y}} $ sur le batch ($ n\times K $),  
- la matrice $ \mathbf{H} $ des activations de la couche cachée ($ n\times L $),  


pour un batch de $ n $ exemples.

#### 📌 Formules à implementer

| **Étape** | **Formule** |
|-----------|------------|
| **Couche cachée (sigmoïde)** | $$ H = \sigma(X W_h + b_h) $$ avec $$ \sigma(x) = \frac{1}{1 + e^{-x}} $$ |
| **Sortie (softmax)** | $$ Y_{\text{pred}} = \text{softmax}(H W_y + b_y) $$ |

In [123]:
def sigmoid(x):
    """ Fonction d'activation sigmoïde """
    sigmoid = 1 / (1 + np.exp(-x))
    return sigmoid

def forward(batch, Wh, bh, Wy, by):
    """ 
    Propagation avant avec activation sigmoïde dans la couche cachée

    Entrées :
    - batch: un batch de n images de MNIST (n, 784)
    - Wh: matrice des poids entrée -> couche cachée (784, hidden_size)
    - bh: biais de la couche cachée (1, hidden_size)
    - Wy: matrice des poids couche cachée -> sortie (hidden_size, 10)
    - by: biais de la sortie (1, 10)

    Renvoie :
    - Y_pred: prédictions de sortie (n, 10)
    - H: activations de la couche cachée (n, hidden_size)
    """

    # ---- PROPAGATION AVANT ----
    # 1️⃣ Calcul de l'activation de la couche cachée
    H = np.matmul(batch, Wh) + bh # Produit matriciel + biais
    H = sigmoid(H) * H  # Application de la sigmoïde

    # 2️⃣ Calcul de l'activation de la sortie
    logits = np.matmul(H, Wy) + by  # Produit matriciel + biais
    exp_logits = np.exp(logits - np.max(logits))  # appliquer la stabilité numérique
    Y_pred = exp_logits / np.sum(exp_logits, axis=1, keepdims=True)  # Softmax

    return Y_pred, H


In [124]:
np.dot(X_train, Wh)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## Apprentissage du perceptron (*backward*)

Comme pour la régression logistique, nous allons entraîner le perceptron à l’aide de l’algorithme
de descente de gradient. Pour calculer les gradients par rapport aux paramètres de la couche cachée,
nous allons avoir besoin d’utiliser l’algorithme de rétro-propagation du gradient (*backpropagation*).
Rappellons que pour chaque batch d’exemples, l’algorithme effectue une passe `forward` (implémentée ci-dessus)
qui permet de calculer la prédiction du perceptron pour les exemples du batch.

La fonction de coût considérée sera encore l’entropie croisée entre la sortie prédite et les étiquettes de
supervision. On calculera alors le gradient de l’erreur par rapport à tous les paramètres du modèle,
c’est-à-dire:

- $ \mathbf{W^y} $ (dimensions $ (L, K) $),  
- $ \mathbf{b^y} $ (dimensions $ (1, K) $),  
- $ \mathbf{W^h} $ (dimensions $ (784, L) $),  
- $ \mathbf{b^h} $ (dimensions $ (1, L) $).  


On rappelle ci-dessous les équations des gradients, effectuées depuis la sortie
vers l’entrée du réseau :

### Etape 1

1. Mise à jour de $ \mathbf{W^y} $ et $ \mathbf{b^y} $:  


$$
\frac{\partial \mathcal{L}}{\partial \mathbf{v_i}} = \mathbf{\delta^y_i} = \mathbf{\hat{y_i}} - \mathbf{y_i^*}
$$

$$
\frac{\partial \mathcal{L}}{\partial \mathbf{W^y}} = \frac{1}{n} \mathbf{H}^T (\mathbf{\hat{Y}} - \mathbf{Y^*}) = \frac{1}{n} \mathbf{H}^T \mathbf{\Delta^y} \tag{1}
$$

$$
\frac{\partial \mathcal{L}}{\partial \mathbf{b^y}} = \frac{1}{n}\sum_{i=1}^{n}(\mathbf{\hat{y_i}} - \mathbf{y_i^*}) \tag{2}
$$

où $ \mathbf{H} $ est la matrice des couches cachées sur le batch
($ 784 \times L $), $ \mathbf{\hat{Y}} $ est la matrice
des prédictions sur le batch (taille $ n \times K $),
$ \mathbf{Y^*} $ est la matrice des étiquettes
issues de la supervision (*ground truth*, $ n \times K $) et
$ \mathbf{\Delta^y}=\mathbf{\hat{Y}}-\mathbf{Y^*} $.

1. Mise à jour de $ \mathbf{W^h} $et $ \mathbf{b^h} $:  

### Etape 2

Les gradients de $ \mathcal{L} $ par rapport à $ \mathbf{W^h} $ et $ \mathbf{b^h} $ s’écrivent matriciellement sous la forme:

$$
    \frac{\partial \mathcal{L}}{\partial \mathbf{u_i}} = \mathbf{\delta^h_i} = \mathbf{\delta^y_i} \mathbf{W^{y}}^T   \odot \sigma^{'}(\mathbf{u_i}) = \mathbf{\delta^y_i} \mathbf{W^{y}}^T \odot \mathbf{h_i} \odot (1-\mathbf{h_i})
$$

$$
\frac{\partial \mathcal{L}}{\partial \mathbf{W^h}} =  \frac{1}{n} \mathbf{X}^T \mathbf{\Delta^h}
~~~\text{et}~~~
\frac{\partial \mathcal{L}}{\partial \mathbf{b^h}} = \frac{1}{n}\sum_{i=1}^{n}(\delta^h_i)
$$

où $ \mathbf{X} $ est la matrice des données sur le batch ($ n \times 784 $) et $ \mathbf{\Delta^h} $ est la matrice des $ \delta^h_i $ sur le batch ($ n \times L $).

## Question

Compléter la fonction `backward` ci-dessous qui calcule et renvoie les gradients de l’erreur par rapport aux paramètres du perceptron.

In [125]:
def backward(Y_pred, Y, X, H, Wy):
    """ Entrées:
    - Y_pred: batch de vecteur des prédictions (one-hot)
    - Y: batch de vecteur des étiquettes (one-hot)
    - X: batch d'images (au format vectoriel (n, 784))
    - H: matrice des activations cachées
    - Wy: matrice des poids

    Renvoie:
    - gradWy: gradient de l'erreur (entropie croisée) par rapport à Wy
    - gradby: gradient de l'erreur (entropie croisée) par rapport à by
    - gradWh: gradient de l'erreur (entropie croisée) par rapport à Wh
    - gradbh: gradient de l'erreur (entropie croisée) par rapport à bh

    """
    delta_y = Y_pred - Y
    # Gradient pour la couche de sortie (identique à la régression logistique)
    gradWy = np.matmul(H.T, delta_y) / Y.shape[0]
    gradby = np.sum(delta_y, axis=0) / Y.shape[0]
    # Gradient pour la couche cachée
    delta_h = np.matmul(delta_y, Wy.T) * (H > 0)
    gradWh = np.matmul(X.T, delta_h) / Y.shape[0]
    gradbh = np.sum(delta_h, axis=0) / Y.shape[0]
    
    return gradWy, gradby, gradWh, gradbh

## Question

La fonction de coût de l’Eq. [(3)](tpDeepLearning1.ipynb#equation-ce) est-elle convexe par
rapport aux paramètres $ \mathbf{W} $, $ \mathbf{b} $ du modèle ? Avec un pas de gradient bien choisi, peut-on assurer la
convergence vers le minimum global de la solution ?

## Question

Compléter le code ci-dessous de sorte à appliquer la descente de gradient sur le perceptron multi-couche défini par les paramètres `Wy, Wh, by, bh`.

In [126]:
import numpy as np
from sklearn.metrics import accuracy_score

N, d = X_train.shape # N exemples, dimension d
hidden_size = 100 # Nombre de neurones de la couche cachée
# Initialisation des poids et des biais
Wy = np.zeros((hidden_size, n_classes))
Wh = np.zeros((d, hidden_size))
by = np.zeros((1, n_classes))
bh = np.zeros((1, hidden_size))

n_epochs = 30 # Nombre d'epochs de la descente de gradient
eta = 1e-1 # Learning rate (pas d'apprentissage)
batch_size = 128 # Taille du lot
n_batches = int(float(N) / batch_size)

# Allocation des matrices pour stocker les valeurs des gradients
gradWy = np.zeros((hidden_size, n_classes))
gradWh = np.zeros((d, hidden_size))
gradby = np.zeros((1, n_classes))
gradbh = np.zeros((1, hidden_size))

for epoch in range(n_epochs):
    print(f"Processing epoch : {epoch}")
    for batch_idx in range(n_batches):
        # ********* À compléter **********              
        X = X_train[batch_idx*batch_size:(batch_idx+1)*batch_size]
        Y = Y_train[batch_idx*batch_size:(batch_idx+1)*batch_size]
        # FORWARD PASS : calculer la prédiction y à partir des paramètres courants pour les images du batch
        Y_pred, H = forward(X, Wh, bh, Wy, by)
        # BACKWARD PASS :
        # 1) calculer les gradients de l'erreur par rapport à W et b
        gradWy, gradby, gradWh, gradbh = backward(Y_pred, Y, X, H, Wy)
        # 2) mettre à jour les paramètres W et b selon la descente de gradient
        Wy = Wy - eta * gradWy
        by = by - eta * gradby
        Wh = Wh - eta * gradWh
        bh = bh - eta * gradbh
        
    
    pred_train = forward(X_train, Wh, bh, Wy, by)[0].argmax(axis=1)
    pred_test = forward(X_test, Wh, bh, Wy, by)[0].argmax(axis=1)
        

    print(f"Epoch {epoch}/{n_epochs}, accuracy (train) = {accuracy_score(pred_train,Y_train.argmax(axis=1))*100:.2f}, accuracy (test) = {accuracy_score(pred_test, Y_test.argmax(axis=1))*100:.2f}")
          
          

Processing epoch : 0
Epoch 0/30, accuracy (train) = 11.24, accuracy (test) = 11.35
Processing epoch : 1
Epoch 1/30, accuracy (train) = 11.24, accuracy (test) = 11.35
Processing epoch : 2
Epoch 2/30, accuracy (train) = 11.24, accuracy (test) = 11.35
Processing epoch : 3
Epoch 3/30, accuracy (train) = 11.24, accuracy (test) = 11.35
Processing epoch : 4
Epoch 4/30, accuracy (train) = 11.24, accuracy (test) = 11.35
Processing epoch : 5
Epoch 5/30, accuracy (train) = 11.24, accuracy (test) = 11.35
Processing epoch : 6
Epoch 6/30, accuracy (train) = 11.24, accuracy (test) = 11.35
Processing epoch : 7
Epoch 7/30, accuracy (train) = 11.24, accuracy (test) = 11.35
Processing epoch : 8
Epoch 8/30, accuracy (train) = 11.24, accuracy (test) = 11.35
Processing epoch : 9
Epoch 9/30, accuracy (train) = 11.24, accuracy (test) = 11.35
Processing epoch : 10
Epoch 10/30, accuracy (train) = 11.24, accuracy (test) = 11.35
Processing epoch : 11
Epoch 11/30, accuracy (train) = 11.24, accuracy (test) = 11.35


## Question

Tester deux autres initialisations :

- initialiser les poids avec une loi normale de moyenne nulle et d’écart type à déterminer, par exemple $ 10^{-1} $,  

##### Utilise le package sklearn pour entraîner un MLP (avec la même architecture que le réseau précédent). Évalue ensuite le modèle et compare les résultats obtenus avec ceux du modèle précédent.